In [1]:
import numpy as np
import pandas as pd
import h5py
from pprint import pprint

from pycmlh5.io import read_and_check_metadata

For now I use the abbreviation **CML** for commercial microwave link. Hence the preliminary name of the file format is **`cmlh5`**, since it is just a definition of the structure of a HDF5 file. I have chosen the abbreviatoin CML over, e.g. MWL, because it is easier to pronounnce and seperates a bit more from all other MW related stuff. I would be happy to get feedback, not only on the file format, but also on the naming.

# Example structure of a cmlh5 file 

```

/                               RootGroup
/cml_1                          Group for first CML
/cml_1/channel_1 		       Group for first channel
/cml_1/channel_1/rx		     Array of RSL values in dBm 
/cml_1/channel_1/tx		     Array of TSL values in dBm
/cml_1/channel_1/time		   Array of timestamps in POSIX time

/cml_1/channel_2 		       Group for second CML channel_2 /cml_1/channel_2/rx 	
/cml_1/channel_2/tx		
/cml_1/channel_2/time


/cml_2 				         Group for second CML
/cml_2/channel_1 		
/cml_2/channel_1/rx		
/cml_2/channel_1/tx		
/cml_2/channel_1/time		

```


# Overview of the metadata definition for each level
The metadata definitions are stored in CSV files so that they can be easily used as the basis for parsers in other languages

## Metadata at the root level

In [2]:
pd.read_csv('metadata_def_root_level.csv', delimiter=';', index_col=0)

,Units,Type,Mandatory,Description
Metadata name,,,,
file_format,None,string,True,This must always be set to ‘cmlh5’
file_format_version,None,string,True,"examples: ‘0.1, ‘1.2’, ..."
author_name,None,string,False,None
author_email,None,string,False,None


## Metadata  at the CML level

In [3]:
pd.read_csv('metadata_def_cml_level.csv', delimiter=';', index_col=0)

,Units,Type,Mandatory,Description
#Metadata name,,,,
site_a_latitude,Decimal degrees,float32,True,None
site_a_longitude,Decimal degrees,float32,True,None
site_a_altitude,Meter,float32,False,None
site_a_antenna_above_ground,Meter,float32,False,None
site_a_id,None,string,False,None
site_b_latitude,Decimal degrees,float32,True,None
site_b_longitude,Decimal degrees,float32,True,None
site_b_altitude,Meter,float32,False,None
site_b_antenna_above_ground,Meter,float32,False,None


## Metadata at the channel level

In [4]:
pd.read_csv('metadata_def_channel_level.csv', delimiter=';', index_col=0)

,Units,Type,Mandatory,Description
Metadata name,,,,
frequency,GHz,float32,True,None
polarization,"[‘V’, ‘H’, ‘v’, ‘h’]",string,True,None
tx_site,None,string,False,None
rx_site,None,string,False,None
channel_id,None,string,True,None
atpc,"[‘on’, ‘off’]",string,True,None
tx_quantization,dBm,float32,False,None
rx_quantization,dBm,float32,False,None
tx_const,dBm,float32,False,value of the TX level if ATPC is off


## Metadata at the array level

...not implemented yet...

# Check cmlh5 files
Right now, this only checks the metadata at the root-, cml- and channel level. Further checks, e.g. for array-level, timestamp format, ..., will follow.

## Check a valid cmlh5 file
This one was provided by Martin

In [5]:
fn = 'example_data/cml_martin2.h5'

cml_metadata_list, error_list = read_and_check_metadata(fn)

In [6]:
pprint(error_list)

[]


In [7]:
pprint(cml_metadata_list)

[{u'cml_1': {u'channel_1': {'metadata': {'additional_info': 'This is a virtual channel of a virtual link generated for testing the CMLh5 data format',
                                         'atpc': 'on',
                                         'channel_id': '14850_10500',
                                         'frequency': 38.5,
                                         'polarization': 'V',
                                         'rx_quantization': nan,
                                         'rx_site': 'site_b',
                                         'tx_quantization': 1.0,
                                         'tx_site': 'site_a'}},
             'metadata': {'cml_id': '14050_10500',
                          'length': 0.89899999,
                          'site_a_latitude': 50.13884,
                          'site_a_longitude': 14.50976,
                          'site_b_latitude': 50.130772,
                          'site_b_longitude': 14.5089}}}]


## Check a file which has some missing metadata and wrong metadata types


The metadata will be parsed, even though their are errors. This behavior could of course be changed, e.g. so that the check simply aborts at the first error.

In [33]:
fn = 'example_data/invalid.h5'

cml_metadata_list, error_list = read_and_check_metadata(fn)

In [35]:
pprint(error_list[0:20])

[u"/cml_0: Metadata `site_a_latitude` is `47.93` with type `<type 'numpy.float64'>` which should be np.float32",
 u"/cml_0: Metadata `site_b_longitude` is `11.29` with type `<type 'numpy.float64'>` which should be np.float32",
 u"/cml_0: Metadata `length` is `7.63541519629` with type `<type 'numpy.float64'>` which should be np.float32",
 u"/cml_0: Metadata `site_b_latitude` is `47.99` with type `<type 'numpy.float64'>` which should be np.float32",
 u'/cml_0: Mandatory metadata `cml_id` is missing',
 u"/cml_0: Metadata `site_a_longitude` is `11.34` with type `<type 'numpy.float64'>` which should be np.float32",
 u'/cml_0/channel_1: Mandatory metadata `channel_id` is missing',
 u'/cml_0/channel_1: Mandatory metadata `polarization` is missing',
 u"/cml_0/channel_1: Metadata `frequency` is `18.085` with type `<type 'numpy.float64'>` which should be np.float32",
 u'/cml_0/channel_1: Mandatory metadata `atpc` is missing',
 u'/cml_0/channel_2: Mandatory metadata `channel_id` is missing',
 u'/

In [10]:
pprint(cml_metadata_list[0:2])

[{u'cml_0': {u'channel_1': {'metadata': {'frequency': 18.085000000000001,
                                         'sampling_type': 'instantaneous',
                                         'temporal_resolution': '1min'}},
             u'channel_2': {'metadata': {'frequency': 19.094999999999999,
                                         'sampling_type': 'instantaneous',
                                         'temporal_resolution': '1min'}},
             'metadata': {'length': 9.9664763732756949,
                          'site_a_latitude': 47.773000000000003,
                          'site_a_longitude': 10.7721,
                          'site_b_latitude': 47.819899999999997,
                          'site_b_longitude': 10.885899999999999,
                          'system_manufacturer': 'Ericsson',
                          'system_model': 'MINI LINK Traffic Node'}}},
 {u'cml_1': {u'channel_1': {'metadata': {'frequency': 18.195,
                                         'sampling_ty

## Let's be less strict with the type checking
e.g. we may not care whether something is a float64 instead of float32

In [36]:
fn = 'example_data/invalid.h5'

cml_metadata_list, error_list = read_and_check_metadata(fn, strict_type_check=False)

In [37]:
pprint(error_list[0:20])

[u'/cml_0: Mandatory metadata `cml_id` is missing',
 u'/cml_0/channel_1: Mandatory metadata `channel_id` is missing',
 u'/cml_0/channel_1: Mandatory metadata `polarization` is missing',
 u'/cml_0/channel_1: Mandatory metadata `atpc` is missing',
 u'/cml_0/channel_2: Mandatory metadata `channel_id` is missing',
 u'/cml_0/channel_2: Mandatory metadata `polarization` is missing',
 u'/cml_0/channel_2: Mandatory metadata `atpc` is missing',
 u'/cml_1: Mandatory metadata `cml_id` is missing',
 u'/cml_1/channel_1: Mandatory metadata `channel_id` is missing',
 u'/cml_1/channel_1: Mandatory metadata `polarization` is missing',
 u'/cml_1/channel_1: Mandatory metadata `atpc` is missing',
 u'/cml_1/channel_2: Mandatory metadata `channel_id` is missing',
 u'/cml_1/channel_2: Mandatory metadata `polarization` is missing',
 u'/cml_1/channel_2: Mandatory metadata `atpc` is missing',
 u'/cml_2: Mandatory metadata `cml_id` is missing',
 u'/cml_2/channel_1: Mandatory metadata `channel_id` is missing',
 u